In [ ]:
import requests
import folium
from datetime import datetime
from IPython.display import display

In [ ]:
# TomTom API Key
TOMTOM_API_KEY = "egk3wuHeRJl9qkbNro85ue43GaKiRj23"

In [ ]:
import requests
import folium
from datetime import datetime
from IPython.display import display

# TomTom API Key
TOMTOM_API_KEY = "egk3wuHeRJl9qkbNro85ue43GaKiRj23"

# Function: Get coordinates using geocoding
def get_coordinates(address):
    url = f"https://api.tomtom.com/search/2/geocode/{address}.json?key={TOMTOM_API_KEY}"
    response = requests.get(url)
    data = response.json()
    try:
        lat = data['results'][0]['position']['lat']
        lon = data['results'][0]['position']['lon']
        return lat, lon
    except (IndexError, KeyError):
        return None

# Function: Get route info and full path
def get_route(start_coords, end_coords):
    start = f"{start_coords[0]},{start_coords[1]}"
    end = f"{end_coords[0]},{end_coords[1]}"
    url = f"https://api.tomtom.com/routing/1/calculateRoute/{start}:{end}/json?key={TOMTOM_API_KEY}&traffic=true"
    response = requests.get(url)
    data = response.json()
    try:
        summary = data['routes'][0]['summary']
        travel_time = summary['travelTimeInSeconds'] / 60  # minutes
        distance = summary['lengthInMeters'] / 1000  # km

        # Extract full route path
        legs = data['routes'][0]['legs'][0]['points']
        route_points = [(point['latitude'], point['longitude']) for point in legs]

        return travel_time, distance, route_points
    except (IndexError, KeyError):
        return None

# Function: Predict traffic level
def predict_traffic_level(travel_time):
    if travel_time <= 15:
        return "Low", "🟢"
    elif travel_time <= 30:
        return "Medium", "🟠"
    else:
        return "High", "🔴"

# Input from user
source = input("Enter Source Address (e.g., Manish Nagar, Nagpur): ")
destination = input("Enter Destination Address (e.g., Dharampeth, Nagpur): ")
time_str = input("Enter Time (HH:MM): ")

# Process input
start_coords = get_coordinates(source)
end_coords = get_coordinates(destination)

if not start_coords or not end_coords:
    print("Error: Could not geocode one or both addresses.")
else:
    route_info = get_route(start_coords, end_coords)
    if not route_info:
        print("Error: Could not retrieve route data.")
    else:
        travel_time, distance, route_path = route_info
        traffic_level, traffic_icon = predict_traffic_level(travel_time)

        print("\n--- Traffic Prediction Result ---")
        print(f"Source: {source}")
        print(f"Destination: {destination}")
        print(f"Time Entered: {time_str}")
        print(f"Estimated Travel Time: {travel_time:.2f} mins")
        print(f"Distance: {distance:.2f} km")
        print(f"Predicted Traffic Level: {traffic_level} {traffic_icon}")

        # Display map in Colab with actual route
        m = folium.Map(location=start_coords, zoom_start=13)
        folium.Marker(start_coords, tooltip="Source", icon=folium.Icon(color="green")).add_to(m)
        folium.Marker(end_coords, tooltip="Destination", icon=folium.Icon(color="red")).add_to(m)
        folium.PolyLine(route_path, color="blue", weight=5).add_to(m)
        display(m)


Enter Source Address (e.g., Manish Nagar, Nagpur): Manish Nagar, Nagpur
Enter Destination Address (e.g., Dharampeth, Nagpur): Wathoda, Nagpur
Enter Time (HH:MM): 09:15

--- Traffic Prediction Result ---
Source: Manish Nagar, Nagpur
Destination: Wathoda, Nagpur
Time Entered: 09:15
Estimated Travel Time: 24.40 mins
Distance: 9.77 km
Predicted Traffic Level: Medium 🟠
